# StressGait Analysis - Self-Reports

## Setup and Helper Functions

In [ ]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import seaborn as sns
from biopsykit.questionnaires.utils import compute_scores, wide_to_long
from biopsykit.stats import StatsPipeline
from fau_colors.v2021 import register_cmaps

from stressgait_analysis.dataset import StressGaitDataset

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
register_cmaps()
plt.close("all")
med_light = sns.color_palette("med_light")
med_dark = sns.color_palette("med_dark")
palette = sns.color_palette("faculties_light")
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"


palette

controls_color = sns.color_palette("wiso")[1]
omc_color = sns.color_palette("med_dark")[1]

stressgait_palette = {
    "OMC": omc_color,
    "Control": controls_color
}
stressgait = [controls_color, omc_color]

In [ ]:
deploy_type = "local"

config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
base_path

In [ ]:
export_path = Path("../../exports")
plot_path = export_path.joinpath("plots")
bp.utils.file_handling.mkdirs([export_path, plot_path])

export_path

In [ ]:
dataset = StressGaitDataset(base_path, coarse_condition=True, exclude_missing_data=True)

dataset

In [ ]:
order = ["Pre", "Marker", "Post"]
hue_order = ["Control", "OMC"]

In [ ]:
data = dataset.questionnaire

In [ ]:
quest_dict = {
    "SSSQ-pre": data.filter(like="Pre_SSSQ").columns,
    "SSSQ-marker": data.filter(like="PreMocap_SSSQ").columns,
    "SSSQ-post": data.filter(like="Post_SSSQ").columns,
    "PANAS-pre": data.filter(like="Pre_PANAS").columns,
    "PANAS-marker": data.filter(like="PreMocap_PANAS").columns,
    "PANAS-post": data.filter(like="Post_PANAS").columns,
}

In [ ]:
quest_data = compute_scores(
    data=data,
    quest_dict=quest_dict,
    quest_kwargs={"PANAS-pre": {"language": "german"}, "PANAS-post": {"language": "german"}},
)

quest_data.head()

In [ ]:
panas_data = wide_to_long(quest_data, quest_name="PANAS", levels=["subscale", "time"])
panas_data = panas_data.rename(index=str.capitalize, level="time")
panas_data

In [ ]:
sssq_data = wide_to_long(quest_data, quest_name="SSSQ", levels=["subscale", "time"])
sssq_data = sssq_data.reindex(["Distress", "Total", "SelfEvaluation"], level="subscale")
sssq_data = sssq_data.rename(index=str.capitalize, level="time")
sssq_data

## PANAS

In [ ]:
panas_data_analysis = panas_data.drop("Total", level="subscale")
panas_data_analysis = panas_data_analysis.rename(index={"NegativeAffect": "Negative Affect", "PositiveAffect": "Positive Affect"}, level="subscale")

panas_data_omc = panas_data_analysis.xs("OMC", level="condition")
panas_data_omc.unstack().head()

In [ ]:
pipeline = StatsPipeline(
    steps=[("prep", "normality"), ("test", "mixed_anova"),("posthoc", "pairwise_tests")],
    params={
        "dv": "PANAS",
        "subject": "participant",
        "within": "time",
        "between": "condition",
        "groupby": "subscale",
         "parametric": False,
        "multicomp": {"method": "bonf", "levels": True},
    },
)

pipeline.apply(panas_data_analysis.drop("Marker", level="time"))

# box_pairs, pvalues = pipeline.sig_brackets("posthoc", plot_type="single", x="subscale", subplots=True, stats_effect_type="within")

pipeline.display_results()

In [ ]:
pipeline = StatsPipeline(
    steps=[("prep", "normality"), ("test", "rm_anova"), ("posthoc", "pairwise_tests")],
    params={
        "dv": "PANAS",
        "subject": "participant",
        "within": "time",
        "groupby": "subscale",
        "test__effsize":'np2',
        "parametric": False,
        "multicomp": {"method": "bonf", "levels": True},
    },
)

pipeline.apply(panas_data_omc)

box_pairs, pvalues = pipeline.sig_brackets(
    "posthoc", plot_type="single", x="subscale", subplots=True, stats_effect_type="within"
)

pipeline.display_results()

In [ ]:
fig, axs = plt.subplots(ncols=2)
sns.set_style("white")
for (key, val), ax in zip(panas_data_omc.groupby("subscale"), axs, strict=False):
    bp.plotting.feature_boxplot(
        data=val.reset_index(),
        x="time",
        y="PANAS",
        hue="time",
        order=order,
        ax=ax,
        stats_kwargs={"pairs": box_pairs[key], "pvalues": pvalues[key], "verbose": 0},
        palette=med_dark[:3],
    )

    ax.set_title(key, fontsize=16)
    ax.set_ylabel("PANAS score", fontsize=16)
    ax.set_xlabel("Time of questionnaire", fontsize=16)
    sns.despine()
    sns.swarmplot(data=val.reset_index(), x="time", y="PANAS", ax=ax, hue="time", palette="fau"[:3], size=8)
    [l.remove() for l in fig.legends]
axs[0].set_xticklabels(["Q-pre", "Q-marker", "Q-post"], fontsize=16)
axs[1].set_xticklabels(["Q-pre", "Q-marker", "Q-post"], fontsize=16)
axs[0].set_yticklabels(axs[0].get_yticks(), fontsize=16)
axs[1].set_yticklabels( axs[0].get_yticks(),fontsize=16)

fig.tight_layout()
plt.savefig("../../exports/plots/PANAS.pdf")

In [ ]:
fig, axs = plt.subplots(ncols=2)

handles, labels = None, None
for (key, val), ax in zip(panas_data_analysis.groupby("subscale"), axs, strict=False):
    sns.boxplot(
        data=val.reset_index(), x="time", y="PANAS", hue="condition", order=order, hue_order=["Control", "OMC"], ax=ax
    )
    ax.set_title(key)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend().remove()
    ax.set_ylabel("PANAS Score [A.U.]")
    
fig.legend(handles, labels, ncols=2, loc="upper center")
fig.tight_layout(rect=(0, 0, 1, 0.95))

## SSSQ

In [ ]:
sssq_data_analysis = sssq_data.reindex(["Distress", "SelfEvaluation", "Total"], level="subscale")
sssq_data_analysis = sssq_data_analysis.rename(index={"SelfEvaluation": "Self Evaluation"}, level="subscale")

sssq_data_groups = sssq_data_analysis.drop("Marker", level="time")

In [ ]:
pipeline = StatsPipeline(
    steps=[("prep", "normality"), ("test", "mixed_anova"), ("posthoc", "pairwise_tests")],
    params={
        "dv": "SSSQ",
        "subject": "participant",
        "within": "time",
        "between": "condition",
        "groupby": "subscale",
        "parametric":False,
        "multicomp": {"method": "bonf", "levels": True},
    },
)

pipeline.apply(sssq_data_groups)

pipeline.display_results(sig_only=False)

In [ ]:
sssq_data_omc = sssq_data_analysis.xs("OMC", level="condition")

In [ ]:
pipeline = StatsPipeline(
    steps=[("prep", "normality"), ("test", "rm_anova"), ("posthoc", "pairwise_tests")],
    params={
        "dv": "SSSQ",
        "subject": "participant",
        "within": "time",
        "groupby": "subscale",
        "parametric":False,
        "test__effsize":"np2",
        "multicomp": {"method": "bonf", "levels": True},
    },
)

pipeline.apply(sssq_data_omc)


box_pairs, pvalues = pipeline.sig_brackets(
    "posthoc", plot_type="single", x="subscale", subplots=True, stats_effect_type="within"
)

pipeline.display_results()






In [ ]:
fig, axs = plt.subplots(ncols=2)
sns.set_style("white")
for (key, val), ax in zip(sssq_data_omc.groupby("subscale"), axs, strict=False):
    bp.plotting.feature_boxplot(
        data=val.reset_index(),
        x="time",
        y="SSSQ",
        hue="time",
        order=order,
        ax=ax,
       # stats_kwargs={"pairs": box_pairs.get(key, []), "pvalues": pvalues.get(key, []), "verbose": 0},
        palette=med_dark[:3],
    )
    sns.swarmplot(data=val.reset_index(), x="time", y="SSSQ", ax=ax, hue="time", palette="fau"[:3], size=8)
    [l.remove() for l in fig.legends]
    ax.set_title(key, fontsize=16)
    ax.set_xlabel("Time of questionnaire", fontsize=16)
    ax.set_ylabel("SSSQ score", fontsize=16)
    sns.despine()
axs[0].set_xticklabels(["Q-pre", "Q-marker", "Q-post"], fontsize=16)
axs[1].set_xticklabels(["Q-pre", "Q-marker", "Q-post"], fontsize=16)
axs[0].set_yticklabels(axs[0].get_yticks(), fontsize=16)
axs[1].set_yticklabels( axs[0].get_yticks(),fontsize=16)
fig.tight_layout()
fig.savefig("../../exports/plots/SSSQ.pdf")